In [ ]:
!pip install torch
!pip install peft
!pip install bitsandbytes
!pip install transformers
!pip install trl
!pip install accelerate
!pip install einops
!pip install datasets
!pip install rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
from rich import print
import json

In [ ]:
# Model
base_model_name = "microsoft/phi-2"

# demo dataset
# dataset = load_dataset("prsdm/medquad-phi2-1k", split="train")

In [ ]:
import json

def read_json_file(file_path):
    """
    Reads a JSON file and returns the data as a Python dictionary.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        dict: A dictionary containing the data from the JSON file, or None if an error occurs.
    """
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {file_path}")
        return None

# Example usage:
file_path = '/content/dataset.json'
data = read_json_file(file_path)

if data:
    print(data)

[
    {
        'text': '### Instruction: What score did he obtain in his Bachelor’s program?\n### Assistant: \n He studied
Computer Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n'
    },
    {
        'text': '### Instruction: What score did he obtain in his Master’s program?\n### Assistant: \n He studied 
Data Science during his Master’s at St. Xavier’s College and achieved 77%.\n'
    },
    {
        'text': '### Instruction: What score did he obtain in his Bachelor’s program?\n### Assistant: \n He studied
Computer Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n'
    },
    {
        'text': '### Instruction: Which subject did Mangaleshwaran focus on during his Master’s?\n### Assistant: \n
He studied Data Science during his Master’s at St. Xavier’s College and achieved 77%.\n'
    },
    {
        'text': '### Instruction: What score did he obtain in his Bachelor’s program?\n### Assistant: \n He studied
Computer Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n'
    },
    {
        'text': '### Instruction: What score did he obtain in his Master’s program?\n### Assistant: \n He studied 
Data Science during his Master’s at St. Xavier’s College and achieved 77%.\n'
    },
    {
        'text': '### Instruction: Where did Mangaleshwaran study Bachelor’s?\n### Assistant: \n He studied Computer
Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n'
    },
    {
        'text': "### Instruction: Which institution awarded Mangaleshwaran's degree in Master’s?\n### Assistant: \n
He studied Data Science during his Master’s at St. Xavier’s College and achieved 77%.\n"
    },
    {
        'text': '### Instruction: Which subject did Mangaleshwaran focus on during his Bachelor’s?\n### Assistant: 
\n He studied Computer Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n'
    },
    {
        'text': '### Instruction: Where did Mangaleshwaran study Master’s?\n### Assistant: \n He studied Data 
Science during his Master’s at St. Xavier’s College and achieved 77%.\n'
    },
    {
        'text': '### Instruction: In which college did he complete his Bachelor’s education?\n### Assistant: \n He 
studied Computer Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n'
    },
    {
        'text': '### Instruction: In which college did he complete his Master’s education?\n### Assistant: \n He 
studied Data Science during his Master’s at St. Xavier’s College and achieved 77%.\n'
    },
    {
        'text': "### Instruction: Which institution awarded Mangaleshwaran's degree in Bachelor’s?\n### Assistant: 
\n He studied Computer Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n"
    },
    {
        'text': '### Instruction: What score did he obtain in his Master’s program?\n### Assistant: \n He studied 
Data Science during his Master’s at St. Xavier’s College and achieved 77%.\n'
    },
    {
        'text': '### Instruction: In which college did he complete his Bachelor’s education?\n### Assistant: \n He 
studied Computer Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n'
    },
    {
        'text': "### Instruction: Which institution awarded Mangaleshwaran's degree in Master’s?\n### Assistant: \n
He studied Data Science during his Master’s at St. Xavier’s College and achieved 77%.\n"
    },
    {
        'text': '### Instruction: What score did he obtain in his Bachelor’s program?\n### Assistant: \n He studied
Computer Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n'
    },
    {
        'text': '### Instruction: Which subject did Mangaleshwaran focus on during his Master’s?\n### Assistant: \n
He studied Data Science during his Master’s at St. Xavier’s College and achieved 77%.\n'
    },
    {
        'text': '### Instruction: Where did Mangaleshwaran study Bachelor’s?\n### Assistant: \n He studi

In [ ]:
# Dataset
from datasets import Dataset
dataset = Dataset.from_list(data)

In [ ]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
dataset[0]

{'text': '### Instruction: What score did he obtain in his Bachelor’s program?\n### Assistant: \n He studied Computer Science during his Bachelor’s at St. John’s College and achieved 7.73 CGPA.\n'}

In [ ]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,  # Changed from keyword argument 'model' to positional argument
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": 0}
)
# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2",quantization_config=bnb_config, torch_dtype="auto", trust_remote_code=True)

model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# LoRA configuration
peft_config = LoraConfig(
    r= 64,
    lora_alpha= 16,
    lora_dropout=0.05, #0.1
    bias="none",
    task_type="CAUSAL_LM",
   #target_modules= ["Wqkv", "out_proj"] #["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ]
)

In [ ]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
)

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/1000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Assuming you want to use wandb for logging
import wandb

# Initialize a wandb run
# You can specify a project name and other parameters here
wandb.init(project="ResumeRiser")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: slowery1992 (slowery1992-stanley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Train model
trainer.train()

Step,Training Loss
25,2.343300
50,1.652800
75,0.725500
100,0.693300
125,0.342500
150,0.435100
175,0.161200
200,0.346000
225,0.103700
250,0.221000


TrainOutput(global_step=250, training_loss=0.7024418334960938, metrics={'train_runtime': 409.2782, 'train_samples_per_second': 2.443, 'train_steps_per_second': 0.611, 'total_flos': 839468550144000.0, 'train_loss': 0.7024418334960938})

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
fine_tuned_model_path='/content/drive/MyDrive/fine_tuned_model/Fine_tuned_microsoft_phi-2-ResumeRiser-7B'
# os.makedirs(fine_tuned_model_path, exist_ok=True)

In [ ]:
os.system('rm -rf /content/drive/MyDrive/fine_tuned_model/')

0

In [ ]:
# Saves the model
model.save_pretrained(fine_tuned_model_path)
trainer.save_model(fine_tuned_model_path)  # Saves the model, tokenizer, and training config
tokenizer.save_pretrained(fine_tuned_model_path)  # Explicitly save the tokenizer (optional, as trainer.save_model may include it)

('/content/drive/MyDrive/fine_tuned_model/Fine_tuned_microsoft_phi-2-ResumeRiser-7B/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_model/Fine_tuned_microsoft_phi-2-ResumeRiser-7B/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_model/Fine_tuned_microsoft_phi-2-ResumeRiser-7B/vocab.json',
 '/content/drive/MyDrive/fine_tuned_model/Fine_tuned_microsoft_phi-2-ResumeRiser-7B/merges.txt',
 '/content/drive/MyDrive/fine_tuned_model/Fine_tuned_microsoft_phi-2-ResumeRiser-7B/added_tokens.json',
 '/content/drive/MyDrive/fine_tuned_model/Fine_tuned_microsoft_phi-2-ResumeRiser-7B/tokenizer.json')

# push to hugging face hub

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
repo_id = "Mangal-404/phi-2-ResumeRiser-7B"  # Replace with your Hugging Face username and desired model name
model.push_to_hub(repo_id, private=False)  # Set private=True if you want a private repository
tokenizer.push_to_hub(repo_id, private=False)

adapter_model.safetensors:   0%|          | 0.00/294M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mangal-404/phi-2-ResumeRiser-7B/commit/55bceedcd4afaa6dfc524cc2af73963fa747a93b', commit_message='Upload tokenizer', commit_description='', oid='55bceedcd4afaa6dfc524cc2af73963fa747a93b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mangal-404/phi-2-ResumeRiser-7B', endpoint='https://huggingface.co', repo_type='model', repo_id='Mangal-404/phi-2-ResumeRiser-7B'), pr_revision=None, pr_num=None)

In [ ]:
repo_id = "Mangal-404/phi-2-ResumeRiser-7B"

In [ ]:
!pip install -q transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer from the Hub
model = AutoModelForCausalLM.from_pretrained(
    repo_id,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(repo_id)

# Test inference
input_text = "What tools and libraries does Mangalesh use for data analysis?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# Clear the memory
del model, trainer
torch.cuda.empty_cache()